# Homework 4

Chris Dailey

christopher.dailey@gmail.com



In [1]:
#imports
import re
from mrjob.job import MRJob
from mrjob.step import MRJobStep

## HW 4.0
What is MrJob? How is it different to Hadoop MapReduce? 
What are the mappint_init, mapper_final(), combiner_final(), reducer_final() methods? When are they called?

## HW 4.1
What is serialization in the context of MrJob or Hadoop? 
When it used in these frameworks? 
What is the default serialization mode for input and outputs for MrJob? 


## HW 4.2

Preprocess the Microsoft log data into a format that can be analyzed in parallel

In [12]:
inputFile = open("anonymous-msweb.data", 'r')

outputFile = open("cleanedlog.txt", 'w')

current_user = ""

for line in inputFile.readlines():
    if line[0] in ["A","T","N", "I"]:
        outputFile.write(line)
    elif line[0] == "C":
        current_user = "C," + re.split(',', line)[2].strip()
    elif line[0] == "V":
        outputFile.write(line.strip() + "," + current_user + "\n")
        
outputFile.close()

## HW 4.3
Find the 5 most frequently visited pages using MrJob from the output of 4.2 (i.e., transfromed log file).

In [89]:
%%writefile pagefrequency.py
import re
from mrjob.job import MRJob
from mrjob.step import MRStep
from mrjob.conf import combine_dicts

class MRFrequencyUtility(MRJob):
    
    def step_two_conf(self):
        #orig_jobconf = super(MRFrequencyUtility, self).jobconf()
        new_jobconf= {
            'stream.num.map.output.key.fields' : 2,
            'mapred.output.key.comparator.class':'org.apache.hadoop.mapred.lib.KeyFieldBasedComparator',
            'mapred.text.key.comparator.options':'-k2,2nr'
        }

        #return combine_dicts(orig_jobconf, new_jobconf) 
        return new_jobconf
    
    def mapper_get_pages(self, _, line):
        splits = [x.strip() for x in line.split(",")]
        if splits[0] =='V':
            yield (splits[1], 1)
                    
    def reducer_count_pages(self, page, counts):
        yield  page, sum(counts)
        
    def mapper_sort_counts(self, page, counts):
        yield page, counts
    
    def reducer_sort_counts(self, page, counts):
        for count in counts:
            yield page, count
        
        
        

    def steps(self):
        return [
            MRStep(mapper=self.mapper_get_pages,
                    reducer=self.reducer_count_pages),
            MRStep(mapper=self.mapper_sort_counts,
                   reducer=self.reducer_sort_counts,
                   jobconf=self.step_two_conf())
        ]
    
if __name__ == '__main__':
    MRFrequencyUtility.run()

Overwriting pagefrequency.py


In [90]:
!hdfs dfs -rm /user/ubuntu/mrJobOutput/*
!hdfs dfs -rmdir /user/ubuntu/mrJobOutput
!python pagefrequency.py --jobconf -numReduceTasks=1 cleanedlog.txt --output-dir mrJobOutput -r hadoop \
--hadoop-home /home/ubuntu/hadoop-2.7.1/

16/02/07 22:09:45 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/ubuntu/mrJobOutput/_SUCCESS
16/02/07 22:09:45 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/ubuntu/mrJobOutput/part-00000
no configs found; falling back on auto-configuration
no configs found; falling back on auto-configuration
creating tmp directory /tmp/pagefrequency.ubuntu.20160207.220948.012645
writing wrapper script to /tmp/pagefrequency.ubuntu.20160207.220948.012645/setup-wrapper.sh
Using Hadoop version 2.7.1
Copying local files into hdfs:///user/ubuntu/tmp/mrjob/pagefrequency.ubuntu.20160207.220948.012645/files/

PLEASE NOTE: Starting in mrjob v0.5.0, protocols will be strict by default. It's recommended you run your job with --strict-protocols or set up mrjob.conf as described at https://pythonhosted.org/mrjob/whats-new.html#ready-for-strict-pr

In [69]:
!hdfs dfs -ls /user/ubuntu/mrJobOutput

Found 2 items
-rw-r--r--   1 ubuntu supergroup          0 2016-02-07 20:24 /user/ubuntu/mrJobOutput/_SUCCESS
-rw-r--r--   1 ubuntu supergroup          0 2016-02-07 20:24 /user/ubuntu/mrJobOutput/part-00000


In [71]:
!hdfs dfs -rm /user/ubuntu/mrJobOutput/*
!hdfs dfs -rmdir /user/ubuntu/mrJobOutput

16/02/07 20:26:34 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/ubuntu/mrJobOutput/_SUCCESS
16/02/07 20:26:34 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/ubuntu/mrJobOutput/part-00000


## HW 4.4
Find the most frequent visitor of each page using MrJob and the output of 4.2  (i.e., transfromed log file). In this output please include the webpage URL, webpageID and Visitor ID.

In [93]:
%%writefile maxvisitors.py
import re
from mrjob.job import MRJob
from mrjob.step import MRStep
from mrjob.conf import combine_dicts

class MRMaxVisitors(MRJob):
    
    SORT_VALUES = True
    
    def mapper_get_page_visits(self, _, line):
        splits = [x.strip() for x in line.split(",")]
        if splits[0] =='V':
            yield ((splits[1], splits[4]), 1)
                    
    def reducer_count_page_visits(self, pair, counts):
        yield  pair, sum(counts)
        
    def steps(self):
        return [
            MRStep(mapper=self.mapper_get_page_visits,
                   reducer=self.reducer_count_page_visits)
        ]
    
if __name__ == '__main__':
    MRMaxVisitors.run()

Overwriting maxvisitors.py


In [94]:
!hdfs dfs -rm /user/ubuntu/mrJobOutput/*
!hdfs dfs -rmdir /user/ubuntu/mrJobOutput
!python maxvisitors.py --jobconf -numReduceTasks=1 cleanedlog.txt --output-dir mrJobOutput -r hadoop \
--hadoop-home /home/ubuntu/hadoop-2.7.1/

16/02/08 15:29:23 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/ubuntu/mrJobOutput/_SUCCESS
16/02/08 15:29:23 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/ubuntu/mrJobOutput/part-00000
no configs found; falling back on auto-configuration
no configs found; falling back on auto-configuration
creating tmp directory /tmp/maxvisitors.ubuntu.20160208.152925.978752
writing wrapper script to /tmp/maxvisitors.ubuntu.20160208.152925.978752/setup-wrapper.sh
Using Hadoop version 2.7.1
Copying local files into hdfs:///user/ubuntu/tmp/mrjob/maxvisitors.ubuntu.20160208.152925.978752/files/

PLEASE NOTE: Starting in mrjob v0.5.0, protocols will be strict by default. It's recommended you run your job with --strict-protocols or set up mrjob.conf as described at https://pythonhosted.org/mrjob/whats-new.html#ready-for-strict-protocol

## HW 4.5
n-dimensional k-means

#### HW 4.5A
K=4 uniform random centroid-distributions over the 1000 words

In [7]:
%%writefile Kmeans.py
from numpy import argmin, array, random
from mrjob.job import MRJob
from mrjob.step import MRStep
from itertools import chain
import warnings

warnings.filterwarnings("ignore")


#Calculate find the nearest centroid for data point 
def MinDist(datapoint, centroid_points):
    datapoint = array(datapoint)
    centroid_points = array(centroid_points)
    diff = datapoint - centroid_points 
    diffsq = diff*diff
    # Get the nearest centroid for each instance
    minidx = argmin(list(diffsq.sum(axis = 1)))
    #print "Nearest: " + str(minidx)
    return minidx

#Check whether centroids converge
def stop_criterion(centroid_points_old, centroid_points_new,T):
    oldvalue = list(chain(*centroid_points_old))
    newvalue = list(chain(*centroid_points_new))
    Diff = [abs(x-y) for x, y in zip(oldvalue, newvalue)]
    return Diff
#     Flag = True
#     for i in Diff:
#         if(i>T):
#             Flag = False
#             break
#     return Flag


class MRKmeans(MRJob):
    centroid_points=[]
    k=4    
    def steps(self):
        return [
            MRStep(mapper_init = self.mapper_init,
                      mapper=self.mapper,
                      combiner = self.combiner,
                      reducer=self.reducer)
               ]
    #load centroids info from file
    def mapper_init(self):
        self.centroid_points = [map(float,s.split('\n')[0].split(',')) for s in open("Centroids.txt").readlines()]
        open('Centroids.txt', 'w').close()
    
    
    #load data and output the nearest centroid index and data point 
    def mapper(self, _, line):
        D = (map(float,line.split(',')))
        #yield int(MinDist(D,self.centroid_points)), (D[0],D[1],1)
        yield int(MinDist(D[3:],self.centroid_points)), (D[3:],1)
    
    
    #Combine sum of data points locally
    def combiner(self, idx, inputdata):
        #sumx = sumy = num = 0
        sums = [0]*1000
        num = 0
        for data ,n in inputdata:
            num = num + n
            sums = [x+y for x,y in zip(sums, data)]
        yield idx,(sums,num)

        
        #Aggregate sum for each cluster and then calculate the new centroids
    def reducer(self, idx, inputdata): 
        centroids = []
        num = [0]*self.k 
        for i in range(self.k):
            centroids.append([0]*1000)
        for data, n in inputdata:
            num[idx] = num[idx] + n
            for i in range(len(data)):
                centroids[idx][i] = centroids[idx][i] + data[i]
        for i in range(1000):
            centroids[idx][i] = centroids[idx][i]/num[idx]
        with open('Centroids.txt', 'a') as f:
            for centroid in centroids:
                if sum(centroid) > 0:
                    f.writelines(','.join(str(j) for j in centroid) + '\n')
#            centroid_string = ""
#            for i in range(1000):
#                centroid_string += str(inputdata[0][i])
#                if i < 999:
#                    centroid_string += ", "
#            f.writelines(centroid_string + '\n')
#            f.writelines(','.join(str(j) for j in i) + '\n' for i in centroids)


        yield idx,(centroids[idx])
      
if __name__ == '__main__':
    MRKmeans.run()

Overwriting Kmeans.py


In [10]:
%reload_ext autoreload
%autoreload 2
from numpy import random
from Kmeans import MRKmeans, stop_criterion
import warnings
import re, random as rnd
mr_job = MRKmeans(args=["topUsers_Apr-Jul_2014_1000-words.txt", "--file", "Centroids.txt"])

warnings.filterwarnings("ignore")

#Geneate initial centroids
centroid_points = []
k = 4

word_data = open("topUsers_Apr-Jul_2014_1000-words.txt", 'r')

data_array = []

#this works with a small sample size but could be done just as well with any random sample
#of a larger dataset
for line in word_data.readlines():
    data_array.append([int(x.strip()) for x in re.split(',', line)[3:]])
    
word_data.close()
    
for i in range(k):
    centroid_points.append(data_array[int(rnd.random()*len(data_array))])

with open('Centroids.txt', 'w') as f:
        f.writelines(','.join(str(j) for j in i) + '\n' for i in centroid_points)

        
# Update centroids iteratively
i = 0
while(1):
    # save previous centoids to check convergency
    centroid_points_old = centroid_points[:]
    print "iteration"+str(i)+":\n"
    with mr_job.make_runner() as runner: 
        with warnings.catch_warnings():
            runner.run()
        # stream_output: get access of the output 
        for line in runner.stream_output():
            key,value =  mr_job.parse_output_line(line)
            centroid_points[key] = value
    print "\n"
    i = i + 1
    Diff = stop_criterion(centroid_points_old, centroid_points, 0.01)
#    if(stop_criterion(centroid_points_old,centroid_points,0.01)):
    done = True
    for m in Diff:
        if(m>0.01):
            done = False
    print "Sum of Diff: " + str(sum(Diff))
    if done:
        break
#print "Centroids\n"
#print centroid_points

iteration0:





Sum of Diff: 360260.229493
iteration1:





Sum of Diff: 329919.552024
iteration2:





Sum of Diff: 480202.803201
iteration3:





Sum of Diff: 185327.80206
iteration4:





Sum of Diff: 277292.168453
iteration5:





Sum of Diff: 21165.1685662
iteration6:





Sum of Diff: 33458.9513205
iteration7:





Sum of Diff: 28492.3067543
iteration8:



Sum of Diff: 0.0


In [ ]:
from numpy import argmin, array, random

varx = [5,5,5,5,5]
vary = [[1,1,1,1,1],[2,2,2,2,2],[3,3,3,3,3],[4,4,4,4,4],[5,5,5,5,5]]
varxa = array(varx)
varya = array(vary)
diffs = varxa - varya
print diffs
diffsq = diffs*diffs
print diffsq
print argmin(list(diffsq.sum(axis=1)))

In [109]:
from itertools import chain
for i in chain(varx, vary):
    print i

TypeError: 'int' object is not iterable

In [53]:
from numpy import argmin, array, random

num = [1]*1000
print num[:5]
print len(num)
num2 = [3]*1000
num3 = [4]*1000
num_combined = [num3, num3, num2, num3]

#print num_combined
def MinDist(datapoint, centroid_points):
    datapoint = array(datapoint)
    centroid_points = array(centroid_points)
    diff = datapoint - centroid_points 
    diffsq = diff*diff
    # Get the nearest centroid for each instance
    minidx = argmin(list(diffsq.sum(axis = 1)))
    return minidx

print MinDist(num, num_combined)

[1, 1, 1, 1, 1]
1000
2


In [2]:
int(3.6)

3

In [5]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import scipy.special as sps
